In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load pressure and DOA data for locations 2 and 4
pressure_files = ['/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_1_0_p.csv', '/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_5_0_p.csv']
doa_files = ['/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_1_0_doa.csv', '/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_5_0_doa.csv']

p_w = []
doa_data = []

for i in range(len(pressure_files)):
    df_pressure = pd.read_csv(pressure_files[i], header=None).values.flatten()#.astype(np.float16)
    df_pressure = df_pressure[400:40000]#.astype(np.float16)
    print("PRESSURE SHAPE: ")
    print(df_pressure.shape)
    print(df_pressure[0] )
    print()


    df_doa = pd.read_csv(doa_files[i], header=None).values#.astype(np.float16)
    df_doa = df_doa[400:40000]#.astype(np.float16)
    print("DOA SHAPE: ")
    print(df_doa.shape)
    print(df_doa[0])
    print()

    combined_df = pd.DataFrame({'pressure': df_pressure, 'doa_x': df_doa[:,0], 'doa_y': df_doa[:,1], 'doa_z': df_doa[:,2]})

    # Drop rows with any NaN values and get corresponding indices
    combined_df = combined_df.dropna()

    # Separate pressure data back
    df_pressure = combined_df['pressure'].values
    df_doa = combined_df[['doa_x', 'doa_y', 'doa_z']].values

    p_w.append(df_pressure)
    doa_data.append(df_doa)

doa_data[0] = doa_data[0][0:30000]
doa_data[1] = doa_data[1][0:30000]
p_w[0] = p_w[0][0:30000]
p_w[1] = p_w[1][0:30000]


PRESSURE SHAPE: 
(39600,)
0.0314622997415602

DOA SHAPE: 
(39600, 3)
[-0.30038384 -0.90115151  2.70345452]

PRESSURE SHAPE: 
(39600,)
-0.0164291138301105

DOA SHAPE: 
(39600, 3)
[ 2.29442997e-16 -1.28148124e+00 -2.56296248e+00]



In [3]:
# Create point clouds for positive and negative pressures
PC2 = {'pos': doa_data[0].reshape(-1,3), 'mass': p_w[0], 'n': len(doa_data[0])}
PC4 = {'pos': doa_data[1].reshape(-1,3), 'mass': p_w[1], 'n': len(doa_data[1])}

print("CHECK IF DIMENSIONS ARE RIGHT: ")
print(PC2['pos'].shape)
print(PC2['mass'].shape)
print(PC2['n'])
print()

CHECK IF DIMENSIONS ARE RIGHT: 
(30000, 3)
(30000,)
30000



In [4]:
interp_dist = 0.5
interp_pressure = (1-interp_dist)*PC2['mass']+interp_dist*PC4['mass']
interp_position = (1-interp_dist)*PC2['pos']+interp_dist*PC4['pos']

In [5]:
pd.DataFrame(interp_pressure).to_csv('interp_csv/lin_3_1-5_p.csv', header=False, index=False)
pd.DataFrame(interp_position).to_csv('interp_csv/lin_3_1-5_doa.csv', header=False, index=False)

In [7]:
pot_neg = POT(PC2_neg, PC4_neg, distEx, distTol)
print("REAACHEEEDDDDDDDDDDDDD end of pot neg")

COST MATRIX SHAPE: 
(30000, 30000)
DUMMY COST: 215.54088126856874
mass transported: 12708.336241443649 index: 0
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 24045
Cost coarse at index 0
mass transported: 12849.53997745969 index: 1
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 24530
Cost coarse at index 1
mass transported: 12990.743713475731 index: 2
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 24990
Cost coarse at index 2
mass transported: 13131.947449491772 index: 3
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 25462
Cost coarse at index 3
mass transported: 13273.151185507812 index: 4
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 25971
Cost coarse at index 4
mass transported: 13414.354921523853 index: 5
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 26445
Cost coarse at index 5
mass transported: 13555.558657539894 index: 6
DONE WITH OBTAINING TX ONCE
Number of non-zeros in Tx: 26990
Cost coarse at index 6
mass transported: 13696.7

## END OF POT. INTERPOLATION OF PC BEGINS HERE

In [8]:
k = 0.5

# Get interpolated point clouds for positive and negative pressures
PCk_pos = pot_pos.interpPC(k)
PCk_neg = pot_neg.interpPC(k)

In [9]:
pd.DataFrame(PCk_pos['mass']).to_csv('pressurepos_int2.csv', header=False, index=False)
pd.DataFrame(PCk_pos['pos']).to_csv('doapos_int2.csv', header=False, index=False)

pd.DataFrame(PCk_neg['mass']).to_csv('pressureneg_int2.csv', header=False, index=False)
pd.DataFrame(PCk_neg['pos']).to_csv('doaneg_int2.csv', header=False, index=False)


In [10]:
### combine pc
all_pressures = np.concatenate(PCk_pos['mass'], PCk_neg['mass'])
all_doas = np.concatenate(PCk_pos['pos'], PCk_neg['pos'])

TypeError: only integer scalar arrays can be converted to a scalar index

In [2]:
### RIR
c=243
timeshifts = np.linalg.norm(all_doas, axis=1)/c

time = np.linspace(0,1,100000)
ir = np.zeros_like(time)

for i in range (len(all_pressures)):
    index = int(np.searchsorted(time, timeshifts[i])
    ir[index]+= all_pressures[i]


plt.figure(figsize=(8,6))
plt.plot(time, ir)
plt.title('Ímpulse Response')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

NameError: name 'all_doas' is not defined

In [8]:
import os
#pressure_files = ['/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_1_0_p.csv', '/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_5_0_p.csv']
#doa_files = ['/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_1_0_doa.csv', '/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/ground_5_0_doa.csv']
pressure_files = [os.path.join('/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/', f'ground_{i}_0_p.csv') for i in range (1,29)]
doa_files = [os.path.join('/Users/valeriolorenzoni/Documents/maria/helixon/SRIR40k/', f'ground_{i}_0_doa.csv') for i in range (1,29)]
p_w = []
doa_data = []

for i in range(len(pressure_files)):
    df_pressure = pd.read_csv(pressure_files[i], header=None).values.flatten()#.astype(np.float16)
    df_pressure = df_pressure[400:40000]#.astype(np.float16)
    print("PRESSURE SHAPE: ")
    print(df_pressure.shape)
    print(df_pressure[0] )
    print()


    df_doa = pd.read_csv(doa_files[i], header=None).values#.astype(np.float16)
    df_doa = df_doa[400:40000]#.astype(np.float16)
    print("DOA SHAPE: ")
    print(df_doa.shape)
    print(df_doa[0])
    print()

    combined_df = pd.DataFrame({'pressure': df_pressure, 'doa_x': df_doa[:,0], 'doa_y': df_doa[:,1], 'doa_z': df_doa[:,2]})

    # Drop rows with any NaN values and get corresponding indices
    combined_df = combined_df.dropna()

    # Separate pressure data back
    df_pressure = combined_df['pressure'].values
    df_doa = combined_df[['doa_x', 'doa_y', 'doa_z']].values

    p_w.append(df_pressure[0:30000])
    doa_data.append(df_doa[0:30000])

output_path = '/Users/valeriolorenzoni/Documents/maria/helixon/GT'
for i in range(1,len(pressure_files)+1):
    pressure_all_files = os.path.join(output_path,f'ground_{i}_0_p.csv')
    doa_all_files = os.path.join(output_path,f'ground_{i}_0_doa.csv')

    pd.DataFrame(p_w[i-1]).to_csv(pressure_all_files, index=False, header=False)
    pd.DataFrame(doa_data[i-1]).to_csv(doa_all_files, index=False, header=False)




PRESSURE SHAPE: 
(39600,)
0.0314622997415602

DOA SHAPE: 
(39600, 3)
[-0.30038384 -0.90115151  2.70345452]

PRESSURE SHAPE: 
(39600,)
0.0494301566846174

DOA SHAPE: 
(39600, 3)
[-1.97215802 -1.97215802 -0.65738601]

PRESSURE SHAPE: 
(39600,)
0.0131101894537361

DOA SHAPE: 
(39600, 3)
[-1.83276663 -1.22184442 -1.83276663]

PRESSURE SHAPE: 
(39600,)
0.0101582681263014

DOA SHAPE: 
(39600, 3)
[-2.75234329 -0.13106397 -0.7863838 ]

PRESSURE SHAPE: 
(39600,)
-0.0164291138301105

DOA SHAPE: 
(39600, 3)
[ 2.29442997e-16 -1.28148124e+00 -2.56296248e+00]

PRESSURE SHAPE: 
(39600,)
-0.0108045368992308

DOA SHAPE: 
(39600, 3)
[ 0.86397448 -0.86397448 -2.59192343]

PRESSURE SHAPE: 
(39600,)
-0.0372648244846545

DOA SHAPE: 
(39600, 3)
[-2.68299735  0.33233888 -0.94976151]

PRESSURE SHAPE: 
(39600,)
-0.0510645552235658

DOA SHAPE: 
(39600, 3)
[2.2478668 0.5619667 1.6859001]

PRESSURE SHAPE: 
(39600,)
-0.0291364171312806

DOA SHAPE: 
(39600, 3)
[ 0.20300096 -2.84201343  0.30450144]

PRESSURE SHAPE: 
